In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix


2025-07-13 14:54:16.182817: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-13 14:54:16.197582: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-13 14:54:16.320887: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-13 14:54:16.472286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752400456.600216   27990 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752400456.65

In [2]:
# Load Dataset
train = pd.read_csv('../data/heartbeat_dataset/mitbih_train.csv', header=None)
test = pd.read_csv('../data/heartbeat_dataset/mitbih_test.csv', header=None)

# Split features and labels
X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values

X_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)

# Reshape for CNN: (samples, time_steps, channels)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [3]:
# Build CNN Model
model = models.Sequential([
    layers.Conv1D(32, 5, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.BatchNormalization(),
    layers.Conv1D(64, 5, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.2),

    layers.Conv1D(128, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling1D(),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/home/mohsin-ali/Desktop/FlutterProjects/heart_disease_prediction_app/backend/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-13 14:56:39.731183: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 183, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 183, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 179, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 179, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 89, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 89, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 87, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 87, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,677 (174.52 KB)

 Trainable params: 44,229 (172.77 KB)

 Non-trainable params: 448 (1.75 KB)

In [4]:
#  Train the Model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 58s 44ms/step - accuracy: 0.9243 - loss: 0.3068 - val_accuracy: 0.1686 - val_loss: 4.9909
Epoch 2/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - accuracy: 0.9654 - loss: 0.1209 - val_accuracy: 0.1169 - val_loss: 8.4738
Epoch 3/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - accuracy: 0.9721 - loss: 0.1006 - val_accuracy: 0.1831 - val_loss: 6.3534
Epoch 4/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - accuracy: 0.9753 - loss: 0.0905 - val_accuracy: 0.1825 - val_loss: 6.8146
Epoch 5/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 56s 45ms/step - accuracy: 0.9770 - loss: 0.0841 - val_accuracy: 0.1731 - val_loss: 8.9058
Epoch 6/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 59s 48ms/step - accuracy: 0.9784 - loss: 0.0801 - val_accuracy: 0.1782 - val_loss: 9.0597
Epoch 7/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - accuracy: 0.9785 - loss: 0.0778 - val_accuracy: 0.1765 - val_loss: 12.1723
Epoch 8/10
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 55s 45ms/step - accuracy: 0.9777 

In [5]:
# Evaluate Model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


685/685 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.9905 - loss: 0.0914
Test Accuracy: 0.8988


In [13]:
import os

# Ensure the model directory exists
os.makedirs('../models/heartbeat_model/saved_model', exist_ok=True)

# Export the model in SavedModel format (needed for TFLite)
model.export('../models/heartbeat_model/saved_model')

# Optionally save in Keras and H5 formats
model.save('../models/heartbeat_model/ecg_model.keras')
model.save('../models/heartbeat_model/model.h5')

# Export to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('../models/heartbeat_model/saved_model')
tflite_model = converter.convert()
with open("../models/heartbeat_model/model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: ../models/heartbeat_model/saved_model/assets


INFO:tensorflow:Assets written to: ../models/heartbeat_model/saved_model/assets


Saved artifact at '../models/heartbeat_model/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 187, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  132450258116560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258118096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258119824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258120208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258117136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258119440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258120016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258119248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258118672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132450258121360: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1752403569.197991   27990 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1752403569.198035   27990 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-13 15:46:09.198368: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ../models/heartbeat_model/saved_model
2025-07-13 15:46:09.201348: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-13 15:46:09.201370: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ../models/heartbeat_model/saved_model
2025-07-13 15:46:09.216013: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-13 15:46:09.281720: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ../models/heartbeat_model/saved_model
2025-07-13 15:46:09.303133: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 10

In [12]:
converter = tf.lite.TFLiteConverter.from_saved_model('../../models/heartbeat_model/saved_model')
tflite_model = converter.convert()

with open("../../models/heartbeat_model/model.tflite", "wb") as f:
    f.write(tflite_model)


W0000 00:00:1752402127.140454   27990 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1752402127.140926   27990 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-13 15:22:07.141499: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ../../models/heartbeat_model/saved_model
2025-07-13 15:22:07.144184: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-13 15:22:07.144197: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ../../models/heartbeat_model/saved_model
I0000 00:00:1752402127.160425   27990 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-13 15:22:07.164536: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-13 15:22:07.275665: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ../../models/heartbeat_model/saved_model
2025-07-13